<a href="https://colab.research.google.com/github/Mar-Ei/Audio/blob/main/Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install aiogram==3.13.1 pyngrok faster-whisper language-tool-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.7/588.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8

In [ ]:
!pip install aiogram==3.13.1 faster-whisper language-tool-python torch soundfile

import nest_asyncio
import asyncio
import tempfile
import os
import language_tool_python
from aiogram import Bot, Dispatcher, types
from aiogram.filters import CommandStart, Command
from faster_whisper import WhisperModel
import torch
from nltk.stem import SnowballStemmer
import re

nest_asyncio.apply()

BOT_TOKEN = "BOT_TOKEN"

device = "cuda" if torch.cuda.is_available() else "cpu"
model = WhisperModel("small", device=device, compute_type="int8")
tool = language_tool_python.LanguageTool('ru-RU')
bot = Bot(token=BOT_TOKEN)
dp = Dispatcher()

HELP_TEXT = "🎤 Отправьте голосовое сообщение или текст."

stemmer = SnowballStemmer("russian")

intents = {
    "greeting": ["привет", "здравствуй", "добрый день", "доброе утро", "хай", "здарова"],
    "thanks": ["спасибо", "благодарю", "thx"],
    "help": ["что ты умеешь", "помощь", "что можешь", "функции", "возможности", "помоги", "help", "хелп"],
    "name": ["как тебя зовут", "кто ты"],
    "joke": ["расскажи шутку", "анекдот", "пошути"]
}

responses = {
    "greeting": "Привет! Отправь мне голосовое сообщение, и я его расшифрую.",
    "thanks": "Пожалуйста! Рад помочь.",
    "help": "Я могу распознавать голосовые сообщения и переводить их в текст с исправлением ошибок, а также отвечать на простые вопросы.",
    "name": "Я — голосовой ассистент, готовый помочь!",
    "joke": "Почему программисты любят темноту? Потому что в темноте меньше багов!",
    "default": "Я пока не понял этот текст. Попробуй отправить голосовое сообщение или спроси что-то из FAQ (/faq)."
}

faq_questions = {
    "Что умеет бот?": responses["help"],
    "Как отправить голосовое?": "Просто отправь мне голосовое сообщение в Telegram.",
    "Как исправляются ошибки?": "После распознавания я исправляю орфографию и делю текст на удобные абзацы."
}

@dp.message(CommandStart())
async def start_cmd(message: types.Message):
    await message.answer(f"Привет, {message.from_user.full_name}!\n{HELP_TEXT}\n\nИспользуй /help или /faq для информации.")

@dp.message(Command(commands=["help"]))
async def help_command(message: types.Message):
    await message.answer("Я могу:\n- Переводить голосовые сообщения в текст\n- Исправлять ошибки\n- Отвечать на простые вопросы\n\nИспользуй /faq для часто задаваемых вопросов.")

@dp.message(Command(commands=["faq"]))
async def faq_command(message: types.Message):
    keyboard_buttons = [
        [types.InlineKeyboardButton(text=question, callback_data=f"faq_{question}")]
        for question in faq_questions.keys()
    ]
    keyboard = types.InlineKeyboardMarkup(inline_keyboard=keyboard_buttons)
    await message.answer("Выберите вопрос:", reply_markup=keyboard)


@dp.callback_query(lambda c: c.data and c.data.startswith("faq_"))
async def faq_callback(callback: types.CallbackQuery):
    key = callback.data.replace("faq_", "")
    answer = faq_questions.get(key, "Информация недоступна.")
    await callback.message.edit_text(answer)
    await callback.answer()

async def download_audio(message: types.Message):
    file_id = message.voice.file_id if message.voice else message.audio.file_id
    file = await bot.get_file(file_id)
    fd, tmp_path = tempfile.mkstemp(suffix=".ogg")
    os.close(fd)
    await bot.download_file(file.file_path, destination=tmp_path)
    return tmp_path

@dp.message(lambda msg: msg.voice or msg.audio)
async def handle_audio(message: types.Message):
    notice = await message.answer("⏳ Обрабатываю аудио...")
    await notice.edit_text("⬇️ Скачиваю аудио...")
    audio_path = await download_audio(message)
    await notice.edit_text("🔍 Распознаю речь...")
    segments, info = model.transcribe(audio_path, language="ru")
    raw_text = " ".join([seg.text for seg in segments])
    if not raw_text.strip():
        await notice.edit_text("⚠️ Не удалось распознать речь.")
        return
    await notice.edit_text("✍️ Исправляю текст...")
    corrected_text = tool.correct(raw_text)
    sentences = corrected_text.split(". ")
    paragraphs = [" ".join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
    final_text = "\n\n".join(paragraphs)
    await notice.edit_text("✅ Готово! Вот текст:\n\n" + final_text[:4000])

def stem_text(text):
    words = re.findall(r'\w+', text.lower())
    return [stemmer.stem(word) for word in words]

@dp.message(lambda msg: msg.text)
async def handle_text(message: types.Message):
    tokens = stem_text(message.text)
    for intent, keywords in intents.items():
        if any(stemmer.stem(word) in tokens for word in keywords):
            await message.answer(responses[intent])
            return
    await message.answer(responses["default"])

async def main():
    print("Бот запущен!")
    await dp.start_polling(bot)

asyncio.run(main())


Бот запущен!


In [3]:

!apt-get update -q
!apt-get install -y openjdk-17-jre
!java -version


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,929 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,520 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/

In [4]:
!pip install --upgrade language-tool-python


In [5]:
import language_tool_python
tool = language_tool_python.LanguageTool('ru-RU')
print("LanguageTool работает!")


INFO:language_tool_python.download_lt:Unzipping /tmp/tmpilp6ws3_.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://internal1.languagetool.org/snapshots/LanguageTool-latest-snapshot.zip to /root/.cache/language_tool_python.


LanguageTool работает!
